In [2]:
import numpy as np
import pandas as pd 
from scipy.optimize import least_squares
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set(rc={"figure.figsize":(25, 10)})

In [110]:
dQI = pd.read_csv("DataForImpact/BIGdQI_10bp.csv")
dQI['Time'] = dQI['Time'].astype(float)
MI = np.array(dQI['Impact'])
dQ= np.array(dQI['DeltaQ'])
T = np.array(dQI['Time'], dtype=float)

### Usual OWM: $I_{t+1} = \rho I_t + \lambda Q_{t+1}$

In [125]:
cvlen = 400
start_point = 100

def fun1(x, mi: np.array, mi_prev: np.array, dq: np.array):
            return x[0] * mi_prev + x[1] * dq - mi

errors01 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[1:i], MI[:i - 1], dQ[1:i]))
    errors01[i - start_point] = fun1(res_lsq01.x, MI[i], MI[i - 1], dQ[i])


print("MAE: ", np.mean(abs(errors01)))

MAE:  5.272029018881983


In [126]:
cvlen = 400
learning_window = 150
start_point = 200

errors02 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq02 = least_squares(fun1, np.array([0.1, 0.1]), args=(MI[i - learning_window:i], MI[i - 1 - learning_window:i - 1], dQ[i - learning_window:i]))
    errors02[i - start_point] = fun1(res_lsq02.x, MI[i], MI[i - 1], dQ[i])

print("MAE: ", np.mean(abs(errors02)))

MAE:  3.651733205706378


### SR model: $I_t = C \sqrt{Q_t}$

In [127]:
cvlen = 400
start_point = 100

def fun2(x, mi: np.array, dq: np.array):
            return x[0] * np.sqrt(dq) - mi

errors03 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq11 = least_squares(fun2, np.array([0.1]), args=(MI[1:i], dQ[1:i]))
    errors03[i - start_point] = fun2(res_lsq11.x, MI[i], dQ[i])


print("MAE: ", np.mean(abs(errors03)))

MAE:  16.575792758682166


In [128]:
cvlen = 400
learning_window = 150
start_point = 200

errors04 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq12 = least_squares(fun2, np.array([0.1, 0.1]), args=(MI[i - learning_window:i], dQ[i - learning_window:i]))
    errors04[i - start_point] = fun2(res_lsq12.x, MI[i], dQ[i])

print("MAE: ", np.mean(abs(errors04)))

MAE:  14.221142403939764


### Experimrntal Combo of SRM and OWM: $I_{t+1} = \rho I_t + \lambda \sqrt{Q_{t+1}}$

In [129]:
cvlen = 400
start_point = 100

def fun3(x, mi: np.array, mi_prev: np.array, dq: np.array):
            return x[0] * mi_prev + x[1] * np.sqrt(dq) - mi

errors01 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun3, np.array([0.1, 0.1]), args=(MI[1:i], MI[:i - 1], dQ[1:i]))
    errors01[i - start_point] = fun3(res_lsq01.x, MI[i], MI[i - 1], dQ[i])


print("MAE: ", np.mean(abs(errors01)))

MAE:  5.272102566367756


In [130]:
cvlen = 400
learning_window = 150
start_point = 200

errors02 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq02 = least_squares(fun3, np.array([0.1, 0.1]), args=(MI[i - learning_window:i], MI[i - 1 - learning_window:i - 1], dQ[i - learning_window:i]))
    errors02[i - start_point] = fun3(res_lsq02.x, MI[i], MI[i - 1], dQ[i])

print("MAE: ", np.mean(abs(errors02)))

MAE:  3.6410765341902724


### Experimental model from OWM: $\frac{I_{i+1} - I_i}{\Delta t _{i+1}} = \rho I_i + \lambda \frac{Q_{i+1}}{\Delta t _{i+1}}$

In [117]:
dt = (T[1:] - T[:-1])
dIdt = (MI[1:] - MI[:-1]) / dt
Qdt = dQ[1:] / dt
Ii = MI[:-1]

In [131]:
cvlen = 400
start_point = 100

def fun3(x, dIdt: np.array, mi_prev: np.array, Qdt: np.array):
            return x[0] * mi_prev + x[1] * Qdt - dIdt

errors01 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun3, np.array([0.1, 0.1]), args=(dIdt[:i], Ii[:i], Qdt[:i]))
    errors01[i - start_point] = Ii[i] * (1 + dt[i] * res_lsq01.x[0]) + res_lsq01.x[1] * Qdt[i] - Ii[i + 1]


print("MAE: ", np.mean(abs(errors01)))

MAE:  19.183232206365943


In [132]:
cvlen = 400
learning_window = 20
start_point = 100

def fun3(x, dIdt: np.array, mi_prev: np.array, Qdt: np.array):
            return x[0] * mi_prev + x[1] * Qdt - dIdt

errors01 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun3, np.array([0.1, 0.1]), args=(dIdt[i - learning_window:i], Ii[i - learning_window:i], Qdt[i - learning_window:i]))
    errors01[i - start_point] = Ii[i] * (1 + dt[i] * res_lsq01.x[0]) + res_lsq01.x[1] * Qdt[i] - Ii[i + 1]


print("MAE: ", np.mean(abs(errors01)))

MAE:  36.430976846660485


### Experimental model from our intuition: $\frac{y_{i + 1} - y_{i}}{\Delta t_{i+1}} = \rho y_i + \lambda$

In [120]:
y = MI / dQ

In [141]:
cvlen = 400
start_point = 100

def fun1(x, y: np.array, y_prev: np.array, dt: np.array):
            return y_prev * (1 + x[0] * dt) + x[1] * dt - y

errors01 = np.full((cvlen - start_point, ), 0.)

for i in range(start_point, cvlen):
    res_lsq01 = least_squares(fun1, np.array([0.1, 0.1]), args=(y[1:i], y[:i - 1], dt[1:i]))
    errors01[i - start_point] = fun1(res_lsq01.x, y[i], y[i - 1], dt[i]) * dQ[i]


print("MAE: ", np.mean(abs(errors01)))

MAE:  22753.029608659912


In [142]:
errors01.astype(int)

array([    -13,  316074,     -13,       0,       0,      20,     -33,
            -4,     -36,    1272,      -1,      69,      41,       7,
             0,      -3,      79,      63,      54,      15,     -17,
            93,      35,      21,      18,      46,     129,     209,
           -20,     230,      57,     -26,     264,     -23,      18,
            58,      88,     -20,     433,      18,     595,       7,
           -12,     170,      12,      40,      17,     225,     210,
           -20,     227,      49,      38,     164,      -4,      74,
            31,      14,      -2,      16,      10,      30,      -2,
        136137,       0,       7,      40,      -2,      19,      17,
            -7,      24,  166136,     -10,       8,  183917,   83988,
          8785,     -25,  105619,       4,      -8,  813674,       7,
            12,      41,       8,      61,       6,      17,      16,
         92516,      -9,       1,      -5,   58764,     -20,      32,
           -16,     

In [137]:
y.astype(int)

array([ 80,  91, 300,  43,  16,   2,   3,   0,   1,   2,   0,   1,   1,
         1,   1,   0,   1,   6,   1,   1,   8,   0,   1,   3,   1,   0,
         0,   1,   0,   2,   1,   9,   1,   0,   0,   0,   0,   1,   0,
         0,   0,   0,   1,   0,  15,   0,   0,   1,  17,  16,   1,   1,
         3,   1,   0,   1,   0,   0,   0,   0,   0,   1,   0,   0,   0,
         0,   4,   2,   3,   3,   1,   3,   0,   2,   0,   1,   7,   0,
         0,   0,   0,   1,   0,   0,   0,   4,   4,   2,   0,   1,   1,
         2,   3,   1,   3,   0,   1,   0,   0,   1,   8,   0,   1,   1,
         1,   1,  28,  30,  61,   3,  20,   3,   3,   4,   6,   8,   1,
         1,   1,   1,  14,   3,   1,   2,   3,   1,   1,   1,   6,   4,
         1,  47,   3,  19,   8,   2,   1,  36,   1,   2,   0,   8,  27,
         3,   3,   3,   1,   0,   2,   8,   0,   1,   1,   1,   8,   3,
         3,   1,   8,   3,   2,   2,   3,   0,   3,   3,   1,   3,   2,
         2,   3,   3,   0,   1,   2,   0,   0,   0,   1,   0,   